In [1]:
import json
from pprint import pprint
import nltk
import re
from sklearn.feature_extraction.text import TfidfVectorizer

In [4]:
data = json.load(open('../data/train-v1.1.json'))

# pprint(data)

In [3]:
pprint(data['data'][0])

{u'paragraphs': [{u'context': u'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.',
                  u'qas': [{u'answers': [{u'answer_start': 515,
                                          u'text': u'Saint Bernadette Soubirous'}],
                            u'id': u'5733be284776f41900661182',
                            u'question': u'To whom did th

                           {u'answers': [{u'answer_start': 488,
                                          u'text': u'39.1%'}],
                            u'id': u'5733ae924776f41900661016',
                            u'question': u'What percentage of students at Notre Dame participated in the Early Action program?'},
                           {u'answers': [{u'answer_start': 618,
                                          u'text': u'more than 750 miles'}],
                            u'id': u'5733ae924776f41900661017',
                            u'question': u'How many miles does the average student at Notre Dame travel to study there?'}]},
                 {u'context': u'In 2015-2016, Notre Dame ranked 18th overall among "national universities" in the United States in U.S. News & World Report\'s Best Colleges 2016. In 2014, USA Today ranked Notre Dame 10th overall for American universities based on data from College Factual. Forbes.com\'s America\'s Best Colleges ranks Notre Dame 13

                           {u'answers': [{u'answer_start': 1193,
                                          u'text': u'Fr. Matthew Walsh'}],
                            u'id': u'573388ce4776f41900660cc7',
                            u'question': u'Which college president of Notre Dame is credited with preventing more confrontations between students and the KKK?'},
                           {u'answers': [{u'answer_start': 819,
                                          u'text': u'a week-long Klavern'}],
                            u'id': u'573388ce4776f41900660cc5',
                            u'question': u'What type of event did the Klan intend to have at Notre Dame in March of 1924?'},
                           {u'answers': [{u'answer_start': 842,
                                          u'text': u'South Bend'}],
                            u'id': u'573388ce4776f41900660cc6',
                            u'question': u'Where did Notre Dame students and the KKK have their encounter?'}

                            u'question': u'What university is Notre Dame tied with in terms of most Heisman Trophy winners?'},
                           {u'answers': [{u'answer_start': 715,
                                          u'text': u'USC'}],
                            u'id': u'5733c4494776f419006611dd',
                            u'question': u'Against which team does Notre Dame compete for the Jeweled Shillelagh?'},
                           {u'answers': [{u'answer_start': 441,
                                          u'text': u'the most'}],
                            u'id': u'5733c4494776f419006611de',
                            u'question': u'In terms of Notre Dame students in the College Football Hall of Fame the amount of students named is what ?'}]},
                 {u'context': u'George Gipp was the school\'s legendary football player during 1916\u201320. He played semiprofessional baseball and smoked, drank, and gambled when not playing sports. He was also humb

In [5]:
#using regex to split sentences

caps = "([A-Z])"
prefixes = "(Mr|St|Mrs|Ms|Dr)[.]"
suffixes = "(Inc|Ltd|Jr|Sr|Co|Rev)"
starters = "(Mr|Mrs|Ms|Dr|He\s|She\s|It\s|They\s|Their\s|Our\s|We\s|But\s|However\s|That\s|This\s|Wherever)"
acronyms = "([A-Z][.][A-Z][.](?:[A-Z][.])?)"
websites = "[.](com|net|org|io|gov|in)"

def split_into_sentences(text):
    text = " " + text + "  "
    text = text.replace("\n"," ")
    text = re.sub(prefixes,"\\1<prd>",text)
    text = re.sub(websites,"<prd>\\1",text)
    if "Ph.D" in text: text = text.replace("Ph.D.","Ph<prd>D<prd>")
    text = re.sub("\s" + caps + "[.] "," \\1<prd> ",text)
    text = re.sub(acronyms+" "+starters,"\\1<stop> \\2",text)
    text = re.sub(caps + "[.]" + caps + "[.]" + caps + "[.]","\\1<prd>\\2<prd>\\3<prd>",text)
    text = re.sub(caps + "[.]" + caps + "[.]","\\1<prd>\\2<prd>",text)
    text = re.sub(" "+suffixes+"[.] "+starters," \\1<stop> \\2",text)
    text = re.sub(" "+suffixes+"[.]"," \\1<prd>",text)
    text = re.sub(" " + caps + "[.]"," \\1<prd>",text)
    if "\"" in text: text = text.replace(".\"","\".")
    if "!" in text: text = text.replace("!\"","\"!")
    if "?" in text: text = text.replace("?\"","\"?")
    text = text.replace(".",".<stop>")
    text = text.replace("?","?<stop>")
    text = text.replace("!","!<stop>")
    text = text.replace("<prd>",".")
    sentences = text.split("<stop>")
    sentences = sentences[:-1]
    sentences = [s.strip() for s in sentences]
    return sentences

In [6]:
vectorizer = TfidfVectorizer(min_df=1)

# print data['data'][0]
complete_tf_idf = {}
for each_set in data['data']:
#     print each_set['title']
    text = []
    for each_para in each_set['paragraphs']:
        if len(text)==0:
            text = split_into_sentences(each_para['context'])
        else:
            text+= split_into_sentences(each_para['context'])
#     print text
    tfidf_data = vectorizer.fit_transform(text)
    idf = vectorizer.idf_
    tf_idf_dict = dict(zip(vectorizer.get_feature_names(), idf))
#     print tf_idf_dict['the']
    complete_tf_idf[each_set['title']]= tf_idf_dict
# print complete_tf_idf['University_of_Notre_Dame']

In [8]:

#extract features from data
training_data = []

for each_para in data['data'][0]['paragraphs']:
    sentence_list = split_into_sentences(each_para['context'])
    for each_q in each_para['qas']:
        data_line = [each_q['id']]
        print(each_q['answers'][0]['text'])
        for each_sent in sentence_list:
            if each_q['answers'][0]['text'] in each_sent:
                print(each_sent)
                sentence_len = len(each_sent.split())
                span_len = len(each_q['answers'][0]['text'].split())
                left_len = len(each_sent[:each_sent.find(each_q['answers'][0]['text'])].split())
                right_len = sentence_len-(span_len+left_len)
                data_line.append(sentence_len)
                data_line.append(span_len)
                data_line.append(left_len)
                data_line.append(right_len)
                tf_idf_sum = 0
                q = each_q['question'].split()
                for each_word in q:
#                     print each_word
                    if each_word in each_sent:
                        try:
                            tf_idf_sum+= complete_tf_idf[data['data'][0]['title']][each_word]
                        except:
                            tf_idf_sum+= 0
                print(tf_idf_sum)
                break
        print(data_line)

Saint Bernadette Soubirous
It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858.
10.943696020202932
['5733be284776f41900661182', 22, 3, 17, 2]
a copper statue of Christ
Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes".
13.22188864224635
['5733be284776f4190066117f', 26, 5, 11, 10]
the Main Building
Atop the Main Building's gold dome is a golden statue of the Virgin Mary.
11.377516920722867
['5733be284776f41900661180', 14, 3, 1, 10]
a Marian place of prayer and reflection
Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection.
7.440171100912844
['5733be284776f41900661181', 14, 7, 7, 0]
a golden statue of the Virgin Mary
Atop the Main Building's gold dome is a golden statue of the Virgin Mary.
11.387841100660731
['5733be284776f4190066117e', 14, 7, 7, 0]
September 1876
Begun as a one-page jour